In [1]:
import os

import nemo
import nemo.collections.asr as nemo_asr

[NeMo W 2023-07-22 02:59:13 experimental:27] Module <class 'nemo.collections.asr.modules.audio_modules.SpectrogramToMultichannelFeatures'> is experimental, not ready for production and is not fully supported. Use at your own risk.


In [2]:
asr_model_path = "./checkpoint/Conformer-CTC-BPE.nemo"
device = 'cpu'
asr_model = nemo_asr.models.EncDecCTCModelBPE.restore_from(asr_model_path, map_location=device)

[NeMo I 2023-07-22 02:59:42 mixins:170] Tokenizer SentencePieceTokenizer initialized with 2048 tokens


[NeMo W 2023-07-22 02:59:52 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /home/work/audrey3/dataset/train_manifest.json
    sample_rate: 16000
    batch_size: 16
    shuffle: true
    num_workers: 7
    pin_memory: true
    max_duration: 20.0
    min_duration: 0.1
    is_tarred: false
    tarred_audio_filepaths: null
    shuffle_n: 2048
    bucketing_strategy: synced_randomized
    bucketing_batch_size: null
    
[NeMo W 2023-07-22 02:59:52 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: /home/work/audrey3/dataset/test_manifest.json
    sample_rate: 16000
    batch_size: 16
    shuffle: fa

[NeMo I 2023-07-22 02:59:52 features:291] PADDING: 0
[NeMo I 2023-07-22 03:00:17 save_restore_connector:249] Model EncDecCTCModelBPE was successfully restored from /home/choisb/Korean-Streaming-ASR/checkpoint/Conformer-CTC-BPE.nemo.


In [ ]:
files = ['./audio_example/0001.wav']
transcript = asr_model.transcribe(paths2audio_files=files)[0]
print(f'Transcript: "{transcript}"')

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcript: "디저트 카페 리뷰가 엄청 좋아서 주문해보려 하는데요 꼬끄 후레이크가 인기가 많던데 정확히 어떤 건가요 후레이크면 시리얼 같은 건가요"


In [ ]:
asr_model.decoder.vocabulary

In [ ]:
lm_path = './checkpoint/kenlm.tmp.arpa'
beam_search_lm = nemo_asr.modules.BeamSearchDecoderWithLM(
    vocab=list(asr_model.decoder.vocabulary),
    beam_width=16,
    alpha=2, beta=1.5,
    lm_path=lm_path,
    num_cpus=max(os.cpu_count(), 1),
    input_tensor=False)